# Feature Engineering Script

**After Second Bus (SB) Data, waze data by the same street and default waze data**

Input variables: 89 + 1 (id)

In [3]:
# importing required packages
import pandas as pd
#import pandas_profiling
import numpy as np
import os
import featuretools as ft

In [4]:
# Choose the city of analysis

# city = 'Recife'
# city = 'CG'
city = 'Curitiba'

In [5]:
# Put the integrated data path

integrated_data_path = os.getcwd() + "/../data/output/" + city + "/"

In [7]:
# Importing the data

output_files = []
days_of_analysis = 0

for dir_name in os.listdir(integrated_data_path):
    
    if dir_name.startswith("output_"): #to get the dir per day
        dir_path = integrated_data_path + dir_name
        days_of_analysis = days_of_analysis+1
        
        for file_name in os.listdir(dir_path):
            
            if file_name.startswith("part-"):
                file_path = dir_path + "/" + file_name
                
                # Join multiple files of the same day
                df = pd.read_csv(file_path, index_col=None, header=0)
                output_files.append(df)
                
print('Days of analysis: ', days_of_analysis)

# Join all output files            
all_integrated_data = pd.concat(output_files, axis=0, ignore_index=True)

all_integrated_data.head(5)

/home/veruska/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (37,82) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/veruska/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (82) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Days of analysis:  31


,route,tripNum,shapeId,routeFrequency,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,...,alertIsJamUnifiedAlertSB,alertInScaleSB,jamUpdateDateTimeSB,jamExpirationDateTimeSB,jamBlockTypeSB,jamDelaySB,jamLengthSB,jamLevelSB,jamSeveritySB,jamSpeedKMSB
0,811,1,2209,high_frequency,4091845,-25.429625,-49.346314,1147.806,BA013,25.0,...,False,False,2019-05-15 05:52:39,NaN,NORMAL,0,0,0,0,NaN
1,811,1,2209,high_frequency,4091845,-25.429625,-49.346314,1147.806,BA015,26.0,...,False,False,2019-05-15 06:13:36,NaN,NORMAL,0,0,0,0,NaN
2,811,2,2209,high_frequency,4091845,-25.429625,-49.346314,1147.806,BA013,364.0,...,False,False,2019-05-15 06:33:37,NaN,NORMAL,0,0,0,0,NaN
3,811,2,2209,high_frequency,4091845,-25.429625,-49.346314,1147.806,BA015,378.0,...,False,False,2019-05-15 06:49:24,NaN,NORMAL,0,0,0,0,NaN
4,811,1,2209,high_frequency,4091845,-25.429625,-49.346314,1147.806,BA006,34.0,...,False,False,2019-05-15 07:04:33,NaN,NORMAL,0,0,0,0,NaN


In [8]:
# Sorting by bus1-bus2-time

all_integrated_data.sort_values(by=['busCode', "busCodeSB", "gps_datetime"], inplace=True)
all_integrated_data.head(5)

,route,tripNum,shapeId,routeFrequency,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,...,alertIsJamUnifiedAlertSB,alertInScaleSB,jamUpdateDateTimeSB,jamExpirationDateTimeSB,jamBlockTypeSB,jamDelaySB,jamLengthSB,jamLevelSB,jamSeveritySB,jamSpeedKMSB
2373477,232,1,1810,high_frequency,6804600,-25.373950,-49.224603,632.186,BA001,NaN,...,False,False,2019-05-27 06:28:10,NaN,NORMAL,0,0,0,0,NaN
2372647,232,1,1810,high_frequency,6804608,-25.372634,-49.227647,976.784,BA001,NaN,...,False,False,2019-05-27 06:28:38,NaN,NORMAL,0,0,0,0,NaN
2375273,232,1,1810,high_frequency,6804617,-25.370722,-49.227966,1236.017,BA001,NaN,...,False,False,2019-05-27 06:29:02,NaN,NORMAL,0,0,0,0,NaN
2376888,232,1,1810,high_frequency,6804636,-25.367622,-49.226984,1594.197,BA001,297.0,...,False,False,2019-05-27 06:29:42,NaN,NORMAL,0,0,0,0,NaN
2374786,232,1,1810,high_frequency,6804651,-25.364842,-49.227710,1911.966,BA001,NaN,...,False,False,2019-05-27 06:30:14,NaN,NORMAL,0,0,0,0,NaN


In [9]:
all_integrated_data.shape

(6211570, 89)

The following variables are objetc type and should be converted to the origin types because the correlation matrix.

In [6]:
all_integrated_data.dtypes

route               int64
tripNum             int64
shapeId             int64
routeFrequency     object
shapeSequence       int64
                   ...   
jamDelaySB          int64
jamLengthSB         int64
jamLevelSB          int64
jamSeveritySB       int64
jamSpeedKMSB      float64
Length: 89, dtype: object

In [10]:
all_integrated_data['shapeId'] = all_integrated_data['shapeId'].astype(str)
all_integrated_data['routeFrequency'] = all_integrated_data['routeFrequency'].astype(str)
all_integrated_data['gps_datetime'] = all_integrated_data['gps_datetime'].astype('datetime64[ns]')
all_integrated_data['problem'] = all_integrated_data['problem'].astype(str)
all_integrated_data['precipitationTime'] = all_integrated_data['precipitationTime'].astype('datetime64[ns]')
all_integrated_data['alertDateTime'] = all_integrated_data['alertDateTime'].astype('datetime64[ns]')
all_integrated_data['alertSubtype'] = all_integrated_data['alertSubtype'].astype(str)
all_integrated_data['alertSubtype'].replace("nan", "-", inplace=True)
all_integrated_data['alertType'] = all_integrated_data['alertType'].astype(str)
all_integrated_data['alertType'].replace("nan", "-", inplace=True)
all_integrated_data['alertIsJamUnifiedAlert'] = all_integrated_data['alertIsJamUnifiedAlert'].astype(bool)
all_integrated_data['alertInScale'] = all_integrated_data['alertInScale'].astype(bool)
all_integrated_data['jamUpdateDateTime'] = all_integrated_data['jamUpdateDateTime'].astype('datetime64[ns]')
all_integrated_data['jamExpirationDateTime'] = all_integrated_data['jamExpirationDateTime'].astype('datetime64[ns]')
all_integrated_data['jamBlockType'] = all_integrated_data['jamBlockType'].astype(str)
all_integrated_data['jamBlockType'].replace("nan", "-", inplace=True)

all_integrated_data['gps_datetimeSB'] = all_integrated_data['gps_datetimeSB'].astype('datetime64[ns]')
all_integrated_data['problemSB'] = all_integrated_data['problemSB'].astype(str)
all_integrated_data['precipitationTimeSB'] = all_integrated_data['precipitationTimeSB'].astype('datetime64[ns]')
all_integrated_data['alertDateTimeSB'] = all_integrated_data['alertDateTimeSB'].astype('datetime64[ns]')
all_integrated_data['alertSubtypeSB'] = all_integrated_data['alertSubtypeSB'].astype(str)
all_integrated_data['alertSubtypeSB'].replace("nan", "-", inplace=True)
all_integrated_data['alertTypeSB'] = all_integrated_data['alertTypeSB'].astype(str)
all_integrated_data['alertTypeSB'].replace("nan", "-", inplace=True)
all_integrated_data['alertIsJamUnifiedAlertSB'] = all_integrated_data['alertIsJamUnifiedAlertSB'].astype(bool)
all_integrated_data['alertInScaleSB'] = all_integrated_data['alertInScaleSB'].astype(bool)
all_integrated_data['jamUpdateDateTimeSB'] = all_integrated_data['jamUpdateDateTimeSB'].astype('datetime64[ns]')
all_integrated_data['jamExpirationDateTimeSB'] = all_integrated_data['jamExpirationDateTimeSB'].astype('datetime64[ns]')
all_integrated_data['jamBlockTypeSB'] = all_integrated_data['jamBlockTypeSB'].astype(str)
all_integrated_data['jamBlockTypeSB'].replace("nan", "-", inplace=True)

#replacing null values
all_integrated_data.jamExpirationDateTime = np.where(all_integrated_data.jamExpirationDateTime.isnull(), 
                                            all_integrated_data.gps_datetime, all_integrated_data.gps_datetime)

all_integrated_data.jamExpirationDateTimeSB = np.where(all_integrated_data.jamExpirationDateTimeSB.isnull(), 
                                            all_integrated_data.gps_datetimeSB, all_integrated_data.gps_datetimeSB)

all_integrated_data.dtypes

route               int64
tripNum             int64
shapeId            object
routeFrequency     object
shapeSequence       int64
                   ...   
jamDelaySB          int64
jamLengthSB         int64
jamLevelSB          int64
jamSeveritySB       int64
jamSpeedKMSB      float64
Length: 89, dtype: object

In [ ]:
# Descriptive statistics
profile = pandas_profiling.ProfileReport(all_integrated_data, check_correlation=True, correlation_threshold=0.5)

In [9]:
profile.to_file(outputfile=integrated_data_path + "output_profile_integrated_data.html")

In [7]:
all_integrated_data['jamExpirationDateTimeSB'].unique()

array(['2019-05-27T06:28:10.000000000', '2019-05-27T06:28:38.000000000',
       '2019-05-27T06:29:02.000000000', ...,
       '2019-05-14T18:08:19.000000000', '2019-05-06T11:29:15.000000000',
       '2019-05-11T16:48:07.000000000'], dtype='datetime64[ns]')

In [9]:
# deleting jamExpirationDateTime because there is no value
all_integrated_data.drop('jamExpirationDateTime', axis=1, inplace=True)
all_integrated_data.drop('jamExpirationDateTimeSB', axis=1, inplace=True)

In [7]:
# saving integrated data before generate new feats
output_path = integrated_data_path + "/" + "integrated_data_before_new_feats.csv"
all_integrated_data.to_csv(output_path, index=False)

* Maybe it didn't measuring correlation because a lot of variables.

### 2. Creating new features


#### 2.1 With Feature Tools

In [ ]:
gps_datetime
precipitationTime
alertDateTime
jamUpdateDateTime
jamExpirationDateTime
gps_datetimeSB
precipitationTimeSB
alertDateTimeSB
jamUpdateDateTimeSB
jamExpirationDateTimeSB

In [18]:
# Creating an entity set
es = ft.EntitySet(id="all_data")

entities = es.entity_from_dataframe(entity_id="id", dataframe=all_integrated_data, make_index=True,
                                    index='id', time_index='gps_datetime')
entities

2020-03-27 15:51:29,738 featuretools.entityset - WARNING    ('Using first column as index. ', 'To change this, specify the index parameter')


OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 3051-01-01 00:00:00

In [11]:
entities.entities

[Entity: id
   Variables:
     id (dtype: index)
     route (dtype: numeric)
     tripNum (dtype: numeric)
     shapeId (dtype: categorical)
     routeFrequency (dtype: categorical)
     shapeSequence (dtype: numeric)
     shapeLat (dtype: numeric)
     shapeLon (dtype: numeric)
     distanceTraveledShape (dtype: numeric)
     busCode (dtype: categorical)
     gpsPointId (dtype: numeric)
     gpsLat (dtype: numeric)
     gpsLon (dtype: numeric)
     distanceToShapePoint (dtype: numeric)
     gps_datetime (dtype: datetime_time_index)
     stopPointId (dtype: numeric)
     problem (dtype: categorical)
     precipitation (dtype: numeric)
     precipitationTime (dtype: datetime)
     precipitationStationDistance (dtype: numeric)
     alertDateTime (dtype: datetime)
     alertSubtype (dtype: categorical)
     alertType (dtype: categorical)
     alertRoadType (dtype: numeric)
     alertConfidence (dtype: numeric)
     alertNComments (dtype: numeric)
     alertNImages (dtype: numeric)
     al

**Para a variável alvo (dados de ônibus por parada de ônibus), podemos gerar as seguintes variáveis:**

In [11]:
all_integrated_data['DAY(alertDateTimeSB)'] = all_integrated_data['alertDateTimeSB'].dt.day
all_integrated_data['DAY(gps_datetime)'] = all_integrated_data['gps_datetime'].dt.day
all_integrated_data['DAY(alertDateTime)'] = all_integrated_data['alertDateTime'].dt.day
all_integrated_data['DAY(jamUpdateDateTime)'] = all_integrated_data['jamUpdateDateTime'].dt.day
all_integrated_data['DAY(jamUpdateDateTimeSB)'] = all_integrated_data['jamUpdateDateTimeSB'].dt.day
all_integrated_data['DAY(precipitationTimeSB)'] = all_integrated_data['precipitationTimeSB'].dt.day
all_integrated_data['DAY(precipitationTime)'] = all_integrated_data['precipitationTime'].dt.day
all_integrated_data['DAY(gps_datetimeSB)'] = all_integrated_data['gps_datetimeSB'].dt.day
all_integrated_data['DAY(jamExpirationDateTime)'] = all_integrated_data['jamExpirationDateTime'].dt.day
all_integrated_data['DAY(jamExpirationDateTimeSB)'] = all_integrated_data['jamExpirationDateTimeSB'].dt.day
all_integrated_data['YEAR(alertDateTimeSB)'] = all_integrated_data['alertDateTimeSB'].dt.year
all_integrated_data['YEAR(gps_datetime)'] = all_integrated_data['gps_datetime'].dt.year
all_integrated_data['YEAR(alertDateTime)'] = all_integrated_data['alertDateTime'].dt.year
all_integrated_data['YEAR(jamUpdateDateTime)'] = all_integrated_data['jamUpdateDateTime'].dt.year
all_integrated_data['YEAR(jamUpdateDateTimeSB)'] = all_integrated_data['jamUpdateDateTimeSB'].dt.year
all_integrated_data['YEAR(precipitationTimeSB)'] = all_integrated_data['precipitationTimeSB'].dt.year
all_integrated_data['YEAR(precipitationTime)'] = all_integrated_data['precipitationTime'].dt.year
all_integrated_data['YEAR(gps_datetimeSB)'] = all_integrated_data['gps_datetimeSB'].dt.year
all_integrated_data['YEAR(jamExpirationDateTime)'] = all_integrated_data['jamExpirationDateTime'].dt.year
all_integrated_data['YEAR(jamExpirationDateTimeSB)'] = all_integrated_data['jamExpirationDateTimeSB'].dt.year
all_integrated_data['MONTH(jamExpirationDateTime)'] = all_integrated_data['jamExpirationDateTime'].dt.month
all_integrated_data['MONTH(jamExpirationDateTimeSB)'] = all_integrated_data['jamExpirationDateTimeSB'].dt.month
all_integrated_data['MONTH(alertDateTimeSB)'] = all_integrated_data['alertDateTimeSB'].dt.month
all_integrated_data['MONTH(gps_datetime)'] = all_integrated_data['gps_datetime'].dt.month
all_integrated_data['MONTH(alertDateTime)'] = all_integrated_data['alertDateTime'].dt.month
all_integrated_data['MONTH(jamUpdateDateTime)'] = all_integrated_data['jamUpdateDateTime'].dt.month
all_integrated_data['MONTH(jamUpdateDateTimeSB)'] = all_integrated_data['jamUpdateDateTimeSB'].dt.month
all_integrated_data['MONTH(precipitationTimeSB)'] = all_integrated_data['precipitationTimeSB'].dt.month
all_integrated_data['MONTH(precipitationTime)'] = all_integrated_data['precipitationTime'].dt.month
all_integrated_data['MONTH(gps_datetimeSB)'] = all_integrated_data['gps_datetimeSB'].dt.month
all_integrated_data['WEEKDAY(alertDateTimeSB)'] = all_integrated_data['alertDateTimeSB'].dt.weekday
all_integrated_data['WEEKDAY(gps_datetime)'] = all_integrated_data['gps_datetime'].dt.weekday
all_integrated_data['WEEKDAY(alertDateTime)'] = all_integrated_data['alertDateTime'].dt.weekday
all_integrated_data['WEEKDAY(jamUpdateDateTime)'] = all_integrated_data['jamUpdateDateTime'].dt.weekday
all_integrated_data['WEEKDAY(jamUpdateDateTimeSB)'] = all_integrated_data['jamUpdateDateTimeSB'].dt.weekday
all_integrated_data['WEEKDAY(precipitationTimeSB)'] = all_integrated_data['precipitationTimeSB'].dt.weekday
all_integrated_data['WEEKDAY(precipitationTime)'] = all_integrated_data['precipitationTime'].dt.weekday
all_integrated_data['WEEKDAY(gps_datetimeSB)'] = all_integrated_data['gps_datetimeSB'].dt.weekday
all_integrated_data['WEEKDAY(jamExpirationDateTime)'] = all_integrated_data['jamExpirationDateTime'].dt.weekday
all_integrated_data['WEEKDAY(jamExpirationDateTimeSB)'] = all_integrated_data['jamExpirationDateTimeSB'].dt.weekday

In [12]:
all_integrated_data.drop(['alertDateTimeSB', 'gps_datetime', 'alertDateTime', 'jamUpdateDateTime', 
                          'jamUpdateDateTimeSB', 'precipitationTimeSB','precipitationTime', 'gps_datetimeSB',
                         'jamExpirationDateTime', 'jamExpirationDateTimeSB'], 
                         axis=1, inplace=True)

,route,tripNum,shapeId,routeFrequency,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,...,MONTH(precipitationTime),MONTH(gps_datetimeSB),WEEKDAY(alertDateTimeSB),WEEKDAY(gps_datetime),WEEKDAY(alertDateTime),WEEKDAY(jamUpdateDateTime),WEEKDAY(jamUpdateDateTimeSB),WEEKDAY(precipitationTimeSB),WEEKDAY(precipitationTime),WEEKDAY(gps_datetimeSB)
2373477,232,1,1810,high_frequency,6804600,-25.373950,-49.224603,632.186,BA001,NaN,...,5,5,0,0,0,0,0,0,0,0
2372647,232,1,1810,high_frequency,6804608,-25.372634,-49.227647,976.784,BA001,NaN,...,5,5,0,0,0,0,0,0,0,0
2375273,232,1,1810,high_frequency,6804617,-25.370722,-49.227966,1236.017,BA001,NaN,...,5,5,0,0,0,0,0,0,0,0
2376888,232,1,1810,high_frequency,6804636,-25.367622,-49.226984,1594.197,BA001,297.0,...,5,5,0,0,0,0,0,0,0,0
2374786,232,1,1810,high_frequency,6804651,-25.364842,-49.227710,1911.966,BA001,NaN,...,5,5,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3892508,23,3,3219,high_frequency,6479247,-25.430278,-49.211046,22831.418,MR802,1316.0,...,5,5,1,1,1,1,1,1,1,1
4045034,23,3,3219,high_frequency,6479263,-25.428790,-49.225490,24294.049,MR802,1326.0,...,5,5,1,1,1,1,1,1,1,1
4053620,23,3,3219,high_frequency,6479293,-25.426009,-49.242224,26104.050,MR802,1347.0,...,5,5,1,1,1,1,1,1,1,1
4148045,23,3,3219,high_frequency,6479361,-25.410513,-49.248234,28715.133,MR802,1388.0,...,5,5,1,1,1,1,1,1,1,1


In [13]:
feature_matrix, features_defs = ft.dfs(entityset=entities, target_entity="id")
feature_matrix.head(5)

,route,tripNum,shapeId,routeFrequency,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,...,MONTH(precipitationTime),MONTH(gps_datetimeSB),WEEKDAY(alertDateTimeSB),WEEKDAY(gps_datetime),WEEKDAY(alertDateTime),WEEKDAY(jamUpdateDateTime),WEEKDAY(precipitationTimeSB),WEEKDAY(jamUpdateDateTimeSB),WEEKDAY(precipitationTime),WEEKDAY(gps_datetimeSB)
id,,,,,,,,,,,,,,,,,,,,,
723,203,1,1943,high_frequency,4812142,-25.444150,-49.265501,9021.842,BE721,956958.0,...,1,1,2,2,2,2,2,2,2,2
724,203,1,1943,high_frequency,4812076,-25.472440,-49.251755,5592.591,BE721,956915.0,...,1,1,2,2,2,2,2,2,2,2
725,203,1,1943,high_frequency,4812093,-25.466695,-49.254369,6282.288,BE721,NaN,...,1,1,2,2,2,2,2,2,2,2
726,203,1,1943,high_frequency,4812108,-25.463026,-49.256323,6735.640,BE721,NaN,...,1,1,2,2,2,2,2,2,2,2
727,203,1,1943,high_frequency,4812119,-25.458404,-49.258586,7296.021,BE721,956877.0,...,1,1,2,2,2,2,2,2,2,2


In [15]:
all_integrated_data.shape

(6211570, 121)

In [14]:
# Saving file
output_path = integrated_data_path + "/" + "new_feats_data.csv"
#feature_matrix.to_csv(output_path, index=False)
all_integrated_data.to_csv(output_path, index=False)

In [14]:
features_defs

[<Feature: route>,
 <Feature: tripNum>,
 <Feature: shapeId>,
 <Feature: routeFrequency>,
 <Feature: shapeSequence>,
 <Feature: shapeLat>,
 <Feature: shapeLon>,
 <Feature: distanceTraveledShape>,
 <Feature: busCode>,
 <Feature: gpsPointId>,
 <Feature: gpsLat>,
 <Feature: gpsLon>,
 <Feature: distanceToShapePoint>,
 <Feature: stopPointId>,
 <Feature: problem>,
 <Feature: precipitation>,
 <Feature: precipitationStationDistance>,
 <Feature: alertSubtype>,
 <Feature: alertType>,
 <Feature: alertRoadType>,
 <Feature: alertConfidence>,
 <Feature: alertNComments>,
 <Feature: alertNImages>,
 <Feature: alertNThumbsUp>,
 <Feature: alertReliability>,
 <Feature: alertReportMood>,
 <Feature: alertReportRating>,
 <Feature: alertSpeed>,
 <Feature: alertLatitude>,
 <Feature: alertLongitude>,
 <Feature: alertDistanceToClosestShapePoint>,
 <Feature: alertIsJamUnifiedAlert>,
 <Feature: alertInScale>,
 <Feature: jamBlockType>,
 <Feature: jamDelay>,
 <Feature: jamLength>,
 <Feature: jamLevel>,
 <Feature: jam

About feature types:
* It works with data type, but generate just mean, sum, count, mode...